# Tutorial 2: IntroducEinführung in PyTorch

Willkommen zu unserem PyTorch-Tutorial für den Deep-Learning-Kurs 2023 an der Universität Amsterdam! Dieses Notebook bietet eine kurze Einführung in die Grundlagen von PyTorch und bereitet dich darauf vor, eigene neuronale Netze zu erstellen. PyTorch ist ein Open-Source-Framework für maschinelles Lernen, das dir flexibles Erstellen und effizientes Optimieren neuronaler Netze ermöglicht. PyTorch ist allerdings nicht das einzige Framework dieser Art. Alternativen sind [TensorFlow](https://www.tensorflow.org/), [JAX](https://github.com/google/jax#quickstart-colab-in-the-cloud) und [Caffe](http://caffe.berkeleyvision.org/). Wir haben uns an der Universität Amsterdam für PyTorch entschieden, da es gut etabliert ist, eine große Entwickler-Community hat (ursprünglich von Facebook entwickelt), sehr flexibel ist und insbesondere in der Forschung eingesetzt wird. Viele aktuelle wissenschaftliche Veröffentlichungen stellen ihren Code in PyTorch bereit, daher ist es von Vorteil, damit vertraut zu sein. TensorFlow (entwickelt von Google) ist hingegen primär als produktionsreife Deep-Learning-Bibliothek bekannt. Wenn du ein Machine-Learning-Framework bereits gut beherrschst, ist es leicht, ein anderes zu erlernen, da viele auf den gleichen Konzepten und Ideen basieren. Zum Beispiel wurde TensorFlow in Version 2 stark von den beliebtesten Features von PyTorch inspiriert, was die Frameworks einander noch ähnlicher gemacht hat. Falls du bereits mit PyTorch vertraut bist und eigene neuronale Netzwerk-Projekte erstellt hast, kannst du dieses Notebook gerne nur überfliegen.

Natürlich sind wir nicht die Ersten, die ein PyTorch-Tutorial erstellen. Es gibt viele großartige Tutorials im Internet, darunter der ["60-min blitz"](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) auf der offiziellen [PyTorch-Webseite](https://pytorch.org/tutorials/). Dennoch haben wir uns entschieden, ein eigenes Tutorial zu gestalten. Dieses soll dir die Grundlagen vermitteln, die du speziell für unsere praktischen Übungen benötigst, und dir trotzdem ein Verständnis der Funktionsweise von PyTorch geben. Im Laufe der nächsten Wochen werden wir auch weiterhin neue PyTorch-Features in dieser Reihe von Jupyter-Notebook-Tutorials rund um Deep Learning erkunden.

Wir werden einen Satz von Standard-Bibliotheken verwenden, die häufig in Machine-Learning-Projekten genutzt werden. Wenn du dieses Notebook auf Google Colab ausführst, sollten alle Bibliotheken vorinstalliert sein. Falls du das Notebook lokal ausführst, stelle sicher, dass du unsere `d12023`-Umgebung installiert und aktiviert hast.

In [1]:
## Standard libraries
import os
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

/var/folders/nj/yxgl90t15y58nn503xvrtr4w0000gn/T/ipykernel_81297/47578708.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


## Die Grundlagen von PyTorch

Wir beginnen mit einem Überblick über die grundlegenden Konzepte von PyTorch. Als Voraussetzung empfehlen wir, mit dem `numpy`-Paket vertraut zu sein, da die meisten Machine-Learning-Frameworks auf sehr ähnlichen Konzepten basieren. Falls du numpy noch nicht kennst, keine Sorge: hier findest du ein [Tutorial](https://numpy.org/devdocs/user/quickstart.html), um dich einzuarbeiten.

Los geht's! Beginnen wir damit, PyTorch zu importieren. Das Paket heißt `torch`, angelehnt an das ursprüngliche Framework Torch. Als ersten Schritt können wir die Version überprüfen:

In [2]:
import torch
print("Using torch", torch.__version__)

Using torch 2.2.1


Zum Zeitpunkt der Erstellung dieses Tutorials (Ende Oktober 2023) ist die aktuelle stabile Version 2.1. Du solltest also als Ausgabe `Using torch 2.1.0` oder `Using torch 2.0.0` sehen, möglicherweise ergänzt durch die Angabe der CUDAVersion auf Colab. Falls du die `d12023`-Umgebung verwendest, solltest du `Using torch 2.1.0` sehen. Generell empfehlen wir, die PyTorch-Version stets auf dem aktuellsten Stand zu halten. Solltest du eine Versionsnummer unter 2.0 sehen, stelle sicher, dass du die richtige Umgebung installiert hast oder frage deine TAs. Im Fall, dass PyTorch 2.2 oder neuer während des Kurses veröffentlicht wird, brauchst du dir keine Sorgen zu machen. Die Schnittstelle zwischen PyTorch-Versionen ändert sich nicht drastisch, daher sollte der Code auch mit neueren Versionen lauffähig sein.

Wie in jedem Machine-Learning-Framework bietet PyTorch Funktionen für zufällige Vorgänge, wie die Erzeugung von Zufallszahlen. Es ist jedoch empfehlenswert, deinen Code so zu gestalten, dass Ergebnisse mit denselben Zufallszahlen reproduzierbar sind. Dies ermöglicht es dir, Fehler aufzuspüren und sicherzustellen, dass Änderungen deinerseits (und nicht zufällige Fluktuationen) die Ursache für veränderte Ergebnisse sind. Deshalb setzen wir nachfolgend einen "Seed" (Startwert). Der Seed ist der Ausgangspunkt für den Zufallszahlengenerator. Durch Setzen eines festen Seeds stellst du sicher, dass die Sequenz der erzeugten Zufallszahlen immer gleich ist.

In [3]:
torch.manual_seed(42) # hier setzen wir den seed

### Tensoren

Tensoren sind das PyTorch-Äquivalent zu NumPy-Arrays, bieten aber zusätzlich Unterstützung für GPU-Beschleunigung (dazu später mehr). Der Name "Tensor" ist eine Verallgemeinerung von Konzepten, die du bereits kennst. Ein Vektor ist zum Beispiel ein 1D-Tensor und eine Matrix ein 2DTensor. Bei der Arbeit mit neuronalen Netzen verwenden wir Tensoren unterschiedlicher Formen und Dimensionen.

Die meisten gängigen Funktionen, die du von NumPy kennst, können auch auf Tensoren angewendet werden. Da NumPy-Arrays und Tensoren sich stark ähneln, können wir die meisten Tensoren in NumPy-Arrays (und umgekehrt) umwandeln. Dies ist jedoch in der Regel nicht oft notwendig.

#### Initialisierung

Fangen wir damit an, uns verschiedene Möglichkeiten zur Erstellung eines Tensors anzusehen. Es gibt viele Optionen, die einfachste ist der Aufruf von `torch.Tensor` mit der gewünschten Form als Eingabeparameter:

In [5]:
x = torch.Tensor(2, 3, 4)
print(x)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


Die Funktion `torch.Tensor` weist den gewünschten Speicher für den Tensor zu, verwendet aber bereits vorhandene Speicherinhalte ggf. weiter. Um dem Tensor direkt bei der Initialisierung Werte zuzuweisen, gibt es verschiedene Möglichkeiten, darunter:
- `torch.zeros`: Erzeugt einen Tensor gefüllt mit Nullen.
- `torch.ones`: Erzeugt einen Tensor gefüllt mit Einsen.
- `torch.rand`: Erzeugt einen Tensor mit Zufallswerten, die gleichmäßig zwischen 0 und 1 verteilt sind.
- `torch.randn`: Erzeugt einen Tensor mit Zufallswerten, die einer Normalverteilung mit Mittelwert 0 und Varianz 1 folgen.
- `torch.arange`: Erzeugt einen Tensor mit den Werten $N, N+1, N+2, \ldots, M$
- `torch.Tensor` (input list): Erzeugt einen Tensor aus einer von dir bereitgestellten Liste.

In [6]:
# Tensor aus (verschachtelter) Liste erzeugen
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

tensor([[1., 2.],
        [3., 4.]])


In [7]:
# Erstelle einen Tensor mit Zufallswerten zwischen 0 und 1 mit der Form [2, 3, 4]
x = torch.rand(2, 3, 4)
print(x)

tensor([[[0.8823, 0.9150, 0.3829, 0.9593],
         [0.3904, 0.6009, 0.2566, 0.7936],
         [0.9408, 0.1332, 0.9346, 0.5936]],

        [[0.8694, 0.5677, 0.7411, 0.4294],
         [0.8854, 0.5739, 0.2666, 0.6274],
         [0.2696, 0.4414, 0.2969, 0.8317]]])


### Dynamische Berechnungsgraphen und Backpropagation

### GPU-Unterstützung/GPU-Beschleunigung

## Lernen durch Beispiele: Kontinuierliches XOR

## Zusätzliche Features, die wir noch nicht besprochen haben